# Data Loading and Exploration

In [ ]:
#Import libraries
import pandas as pd
import dask.dataframe as dd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from functools import reduce
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score

In [2]:
#Load data
df_cropsprod = pd.read_csv("Crops production indicators - FAOSTAT_data_en_2-22-2024.csv")
df_exchange = pd.read_csv("Exchange rate - FAOSTAT_data_en_2-22-2024.csv")
df_foodbalance = pd.read_csv("Food balances indicators - FAOSTAT_data_en_2-22-2024.csv")
df_foodtrade = pd.read_csv("Food trade indicators - FAOSTAT_data_en_2-22-2024.csv")
df_landtemp = pd.read_csv("Land temperature change - FAOSTAT_data_en_2-27-2024.csv")

## Exploring each dataset

In [3]:
dfs = [df_cropsprod, df_exchange, df_foodbalance, df_foodtrade, df_landtemp]
df_names = ['cropsprod', 'exchange', 'foodbalance', 'foodtrade', 'landtemp']

In [4]:
#Check df properties
for df_name, df in zip(df_names, dfs):
    print(f"Exploring DataFrame: {df_name}\n")

    # Explore the DataFrame
    print("Info:")
    print(df.info())
    
    print("n/samples:")
    print(df.sample(5))
    print("\n")
    
    print("Shape:")
    print(df.shape)
    print("\n")
    
    print("Columns:")
    print(df.columns)
    print("\n")

    print("Value counts:")
    print(df.value_counts())
    print("\n")



Exploring DataFrame: cropsprod

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41649 entries, 0 to 41648
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       41649 non-null  object 
 1   Domain            41649 non-null  object 
 2   Area Code (M49)   41649 non-null  int64  
 3   Area              41649 non-null  object 
 4   Element Code      41649 non-null  int64  
 5   Element           41649 non-null  object 
 6   Item Code (CPC)   41649 non-null  object 
 7   Item              41649 non-null  object 
 8   Year Code         41649 non-null  int64  
 9   Year              41649 non-null  int64  
 10  Unit              41649 non-null  object 
 11  Value             41649 non-null  int64  
 12  Flag              41649 non-null  object 
 13  Flag Description  41649 non-null  object 
 14  Note              0 non-null      float64
dtypes: float64(1), int64(5), object(9)
memory usage: 

Domain Code  Domain                 Area Code (M49)  Area         Element Code  Element                Item Code (FBS)  Item                      Year Code  Year  Unit    Value   Flag  Flag Description
FBS          Food Balances (2010-)  4                Afghanistan  5123          Losses                 S2905            Cereals - Excluding Beer  2010       2010  1000 t  837.00  E     Estimated value     1
                                    558              Nicaragua    5154          Other uses (non-food)  S2908            Sugar Crops               2018       2018  1000 t  521.00  E     Estimated value     1
                                                                  5142          Food                   S2949            Eggs                      2014       2014  1000 t  31.00   E     Estimated value     1
                                                                                                                                                  2015       2015  1000 t  33.00 

In [5]:
df_foodtrade.Element.unique()

array(['Import Value', 'Export Value'], dtype=object)

In [6]:
df_foodtrade["Element Code"].unique()

array([5622, 5922])

In [7]:
df_foodbalance["Element Code"].unique()

array([5611, 5911, 5123, 5154, 5142])

In [8]:
df_foodbalance['Element'].unique()

array(['Import Quantity', 'Export Quantity', 'Losses',
       'Other uses (non-food)', 'Food'], dtype=object)

# Data Preprocessing

In [9]:
#average exchange rate per year
avg_exchange_rate_per_year = df_exchange.groupby(['Year', 'Area'])['Value'].mean().reset_index()

In [10]:
avg_exchange_rate_per_year.head()

,Year,Area,Value
0,1980,Afghanistan,44.129167
1,1980,Algeria,3.837450
2,1980,Angola,29.900000
3,1980,Anguilla,2.700000
4,1980,Antigua and Barbuda,2.700000


In [11]:
#average temperature change per year
avg_temp_per_year = df_landtemp.groupby(['Year', 'Area'])['Value'].mean().reset_index()

In [12]:
#average temperature change per year
avg_temp_per_year.head()

,Year,Area,Value
0,2000,Afghanistan,0.9128
1,2000,Albania,0.9002
2,2000,Algeria,0.6867
3,2000,American Samoa,0.4704
4,2000,Andorra,0.8999


In [13]:
#duplicate df_fo
df_foodtrade = df_foodtrade[df_foodtrade['Element Code'] == 5622].copy()

#drop unneccessary columns
df_foodtrade.drop(columns=['Domain Code', 'Domain', 'Item Code (CPC)', 'Area Code (M49)', 'Element Code', 'Element', 'Year Code', 'Unit', 'Flag', 'Flag Description', 'Note'], inplace=True)


In [14]:
df_foodtrade.head()

,Area,Item,Year,Value
0,Afghanistan,Cereals and Preparations,1991,41600.0
1,Afghanistan,Cereals and Preparations,1992,25600.0
2,Afghanistan,Cereals and Preparations,1993,40000.0
3,Afghanistan,Cereals and Preparations,1994,25700.0
4,Afghanistan,Cereals and Preparations,1995,37720.0


In [15]:
#Filter rows in df_foodbalance
df_foodbalance = df_foodbalance[df_foodbalance['Element Code'] == 5911].copy()
df_foodbalance = df_foodbalance[['Year', 'Area', 'Item', 'Value']]

In [16]:
df_foodbalance.head()

,Year,Area,Item,Value
12,2010,Afghanistan,Cereals - Excluding Beer,0.0
13,2011,Afghanistan,Cereals - Excluding Beer,0.0
14,2012,Afghanistan,Cereals - Excluding Beer,0.0
15,2013,Afghanistan,Cereals - Excluding Beer,0.0
16,2014,Afghanistan,Cereals - Excluding Beer,2.0


In [17]:
#Rename columns
df_foodtrade.rename(columns={"Item": "Products", "Value": "Export Value"}, inplace=True)
df_foodbalance.rename(columns={"Item": "Products", "Value": "Export Quantity"}, inplace=True)
avg_exchange_rate_per_year.rename(columns={"Value": "Exchange Rate"}, inplace=True)
avg_temp_per_year.rename(columns={"Value": "Land Temperature"}, inplace=True)
df_cropsprod.rename(columns={"Item": "Products", "Value": "Crop Production"}, inplace=True)

# Feature Engineering

In [18]:
df_cropsprod = df_cropsprod[['Year', 'Area', 'Products', 'Crop Production']]

In [19]:
df_cropsprod.head()

,Year,Area,Products,Crop Production
0,2000,Afghanistan,"Cereals, primary",8063
1,2001,Afghanistan,"Cereals, primary",10067
2,2002,Afghanistan,"Cereals, primary",16698
3,2003,Afghanistan,"Cereals, primary",14580
4,2004,Afghanistan,"Cereals, primary",13348


In [20]:
df = pd.merge(df_foodtrade, df_foodbalance, on=['Year', 'Area', 'Products'], how='outer')
df = pd.merge(df, df_cropsprod, on=['Year', 'Area', 'Products'], how='outer')

In [21]:
df["Crop Production"]

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
147163    66518.0
147164    64830.0
147165    65628.0
147166    66126.0
147167    65856.0
Name: Crop Production, Length: 147168, dtype: float64

In [22]:
df = pd.merge(df, avg_exchange_rate_per_year, on=['Year', 'Area'], how='left')
df = pd.merge(df, avg_temp_per_year, on=['Year', 'Area'], how='left')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147168 entries, 0 to 147167
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Area              147168 non-null  object 
 1   Products          147168 non-null  object 
 2   Year              147168 non-null  int64  
 3   Export Value      73914 non-null   float64
 4   Export Quantity   33676 non-null   float64
 5   Crop Production   41649 non-null   float64
 6   Exchange Rate     139180 non-null  float64
 7   Land Temperature  123657 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 10.1+ MB


In [24]:
#extract and merge relevant features from df_exchange
df_exchange_subset = df_exchange[['Year', 'Area', 'Value']]
df_exchange_subset = df_exchange_subset.rename(columns={'Value': 'Exchange Rate'})
df = pd.merge(df, df_exchange_subset, on=['Year', 'Area'], how='left')

In [25]:
df

,Area,Products,Year,Export Value,Export Quantity,Crop Production,Exchange Rate_x,Land Temperature,Exchange Rate_y
0,Afghanistan,Cereals and Preparations,1991,41600.0,NaN,NaN,50.599605,NaN,50.599605
1,Afghanistan,Cereals and Preparations,1991,41600.0,NaN,NaN,50.599605,NaN,50.599605
2,Afghanistan,Cereals and Preparations,1991,41600.0,NaN,NaN,50.599605,NaN,50.599605
3,Afghanistan,Cereals and Preparations,1991,41600.0,NaN,NaN,50.599605,NaN,50.599605
4,Afghanistan,Cereals and Preparations,1991,41600.0,NaN,NaN,50.599605,NaN,50.599605
...,...,...,...,...,...,...,...,...,...
1680321,Zimbabwe,Vegetables Primary,2022,NaN,NaN,65856.0,349.518396,-0.0091,337.155164
1680322,Zimbabwe,Vegetables Primary,2022,NaN,NaN,65856.0,349.518396,-0.0091,400.810038
1680323,Zimbabwe,Vegetables Primary,2022,NaN,NaN,65856.0,349.518396,-0.0091,495.896171
1680324,Zimbabwe,Vegetables Primary,2022,NaN,NaN,65856.0,349.518396,-0.0091,597.970000


In [26]:
#extract and merge relevant features from df_landtemp
df_landtemp_subset = df_landtemp[['Year', 'Area', 'Value']]
df_landtemp_subset = df_landtemp_subset.rename(columns={'Value': 'Land Temperature'})
df = pd.merge(df, df_landtemp_subset, on=['Year', 'Area'], how='left')

In [27]:
df.isna().sum()

Area                         0
Products                     0
Year                         0
Export Value           8466690
Export Quantity       10856485
Crop Production       10079205
Exchange Rate_x          62384
Land Temperature_x      551525
Exchange Rate_y          62384
Land Temperature_y     1758532
dtype: int64

In [28]:
#Fill Null Values
df["Export Value"] = df.groupby("Area")["Export Value"].transform(lambda x: x.fillna(x.mean()))
df["Export Quantity"] = df.groupby("Area")["Export Quantity"].transform(lambda x: x.fillna(x.mean()))
df["Crop Production"] = df.groupby("Area")["Crop Production"].transform(lambda x: x.fillna(x.mean()))
df["Exchange Rate_x"] = df.groupby("Area")["Exchange Rate_x"].transform(lambda x: x.fillna(x.mean()))
df["Land Temperature_x"] = df.groupby("Area")["Land Temperature_x"].transform(lambda x: x.fillna(x.mean()))
df["Exchange Rate_y"] = df.groupby("Area")["Exchange Rate_y"].transform(lambda x: x.fillna(x.mean()))
df["Land Temperature_y"] = df.groupby("Area")["Land Temperature_y"].transform(lambda x: x.fillna(x.mean()))

In [29]:
df["Export Value"].fillna(df["Export Value"].mean(), inplace=True)
df["Export Quantity"].fillna(df["Export Quantity"].mean(), inplace=True)
df["Crop Production"].fillna(df["Crop Production"].mean(), inplace=True)
df["Exchange Rate_x"].fillna(df["Exchange Rate_x"].mean(), inplace=True)
df["Land Temperature_x"].fillna(df["Land Temperature_x"].mean(), inplace=True)
df["Exchange Rate_y"].fillna(df["Exchange Rate_y"].mean(), inplace=True)
df["Land Temperature_y"].fillna(df["Land Temperature_y"].mean(), inplace=True)

In [30]:
df.dropna(inplace=True)

In [31]:
df.isna().sum()

Area                  0
Products              0
Year                  0
Export Value          0
Export Quantity       0
Crop Production       0
Exchange Rate_x       0
Land Temperature_x    0
Exchange Rate_y       0
Land Temperature_y    0
dtype: int64

In [32]:
#create lag features
df['Export Value Lag 1'] = df.groupby(['Area', 'Products'])['Export Value'].shift(1)
df['Export Value Lag 2'] = df.groupby(['Area', 'Products'])['Export Value'].shift(2)
df['Export Value Lag 3'] = df.groupby(['Area', 'Products'])['Export Value'].shift(3)

df['Export Quantity Lag 1'] = df.groupby(['Area', 'Products'])['Export Quantity'].shift(1)
df['Export Quantity Lag 2'] = df.groupby(['Area', 'Products'])['Export Quantity'].shift(2)

df['Crop Production Lag 1'] = df.groupby(['Area', 'Products'])['Crop Production'].shift(1)
df['Crop Production Lag 2'] = df.groupby(['Area', 'Products'])['Crop Production'].shift(2)

df['Exchange Rate_x Lag 1'] = df.groupby(['Area'])['Exchange Rate_x'].shift(1)
df['Exchange Rate_x Lag 2'] = df.groupby(['Area'])['Exchange Rate_x'].shift(2)

df['Land Temperature_x Lag 1'] = df.groupby(['Area'])['Land Temperature_x'].shift(1)
df['Land Temperature_x Lag 2'] = df.groupby(['Area'])['Land Temperature_x'].shift(2)

df['Exchange Rate_y Lag 1'] = df.groupby(['Area'])['Exchange Rate_y'].shift(1)
df['Exchange Rate_y Lag 2'] = df.groupby(['Area'])['Exchange Rate_y'].shift(2)

df['Land Temperature_y Lag 1'] = df.groupby(['Area'])['Land Temperature_y'].shift(1)
df['Land Temperature_y Lag 2'] = df.groupby(['Area'])['Land Temperature_y'].shift(2)

In [33]:
df.isna().sum()

Area                            0
Products                        0
Year                            0
Export Value                    0
Export Quantity                 0
Crop Production                 0
Exchange Rate_x                 0
Land Temperature_x              0
Exchange Rate_y                 0
Land Temperature_y              0
Export Value Lag 1           7228
Export Value Lag 2          14445
Export Value Lag 3          21650
Export Quantity Lag 1        7228
Export Quantity Lag 2       14445
Crop Production Lag 1        7228
Crop Production Lag 2       14445
Exchange Rate_x Lag 1         212
Exchange Rate_x Lag 2         424
Land Temperature_x Lag 1      212
Land Temperature_x Lag 2      424
Exchange Rate_y Lag 1         212
Exchange Rate_y Lag 2         424
Land Temperature_y Lag 1      212
Land Temperature_y Lag 2      424
dtype: int64

In [34]:
#Fill null values with overall mean
df['Export Value Lag 1'] = df['Export Value Lag 1'].fillna(df['Export Value Lag 1'].mean())
df['Export Value Lag 2'] = df['Export Value Lag 2'].fillna(df['Export Value Lag 2'].mean())
df['Export Value Lag 3'] = df['Export Value Lag 3'].fillna(df['Export Value Lag 3'].mean())
df['Export Quantity Lag 1'] = df['Export Quantity Lag 1'].fillna(df['Export Quantity Lag 1'].mean())
df['Export Quantity Lag 2'] = df['Export Quantity Lag 2'].fillna(df['Export Quantity Lag 2'].mean())
df['Crop Production Lag 1'] = df['Crop Production Lag 1'].fillna(df['Crop Production Lag 1'].mean())
df['Crop Production Lag 2'] = df['Crop Production Lag 2'].fillna(df['Crop Production Lag 2'].mean())
df['Exchange Rate_x Lag 1'] = df['Exchange Rate_x Lag 1'].fillna(df['Exchange Rate_x Lag 1'].mean())
df['Exchange Rate_x Lag 2'] = df['Exchange Rate_x Lag 2'].fillna(df['Exchange Rate_x Lag 2'].mean())
df['Land Temperature_x Lag 1'] = df['Land Temperature_x Lag 1'].fillna(df['Land Temperature_x Lag 1'].mean())
df['Land Temperature_x Lag 2'] = df['Land Temperature_x Lag 2'].fillna(df['Land Temperature_x Lag 2'].mean())
df['Exchange Rate_y Lag 1'] = df['Exchange Rate_y Lag 1'].fillna(df['Exchange Rate_y Lag 1'].mean())
df['Exchange Rate_y Lag 2'] = df['Exchange Rate_y Lag 2'].fillna(df['Exchange Rate_y Lag 2'].mean())
df['Land Temperature_y Lag 1'] = df['Land Temperature_y Lag 1'].fillna(df['Land Temperature_y Lag 1'].mean())
df['Land Temperature_y Lag 2'] = df['Land Temperature_y Lag 2'].fillna(df['Land Temperature_y Lag 2'].mean())

In [35]:
scaler = StandardScaler()
df[['Exchange Rate_x', 'Exchange Rate_y', 'Land Temperature_y', 'Land Temperature_x', 'Crop Production']] = scaler.fit_transform(df[['Exchange Rate_x', 'Exchange Rate_y', 'Land Temperature_y', 'Land Temperature_x', 'Crop Production']])

In [36]:
df.isna().sum()

Area                        0
Products                    0
Year                        0
Export Value                0
Export Quantity             0
Crop Production             0
Exchange Rate_x             0
Land Temperature_x          0
Exchange Rate_y             0
Land Temperature_y          0
Export Value Lag 1          0
Export Value Lag 2          0
Export Value Lag 3          0
Export Quantity Lag 1       0
Export Quantity Lag 2       0
Crop Production Lag 1       0
Crop Production Lag 2       0
Exchange Rate_x Lag 1       0
Exchange Rate_x Lag 2       0
Land Temperature_x Lag 1    0
Land Temperature_x Lag 2    0
Exchange Rate_y Lag 1       0
Exchange Rate_y Lag 2       0
Land Temperature_y Lag 1    0
Land Temperature_y Lag 2    0
dtype: int64

In [47]:
print(df.Year.min())
print(df.Year.max())

1991
2022


In [53]:
data = df.copy()

#Label encoding on categorical variables
encoder = LabelEncoder()
data[['Area', 'Products']] = data[['Area', 'Products']].apply(lambda x: encoder.fit_transform(x))


#Split the data into training and validation sets based on a specific year threshold
train_data = data[data["Year"] <= 2020]
val_data = data[(data["Year"] > 2020) & (data["Year"] <= 2022)]

#Separate features and target variable for training and validation sets
X_train = train_data.drop(columns=['Export Value', 'Year'])
y_train = train_data['Export Value']
X_val = val_data.drop(columns=['Export Value', 'Year'])
y_val = val_data['Export Value']



#training and testing sets sizes
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])

Training set size: 13586325
Testing set size: 1236800


# Model Training

In [55]:
import time


mlp_regressor = MLPRegressor(hidden_layer_sizes=(50,25), activation='relu', solver='adam', max_iter=500, alpha = 0.01, batch_size = 64, random_state=42)
start_time = time.time()
mlp_regressor.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time
print("Training time:", training_time, "seconds")

Training time: 15199.982423782349 seconds


In [56]:
#Model evaluation
y_pred_val = mlp_regressor.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)
mae_val = mean_absolute_error(y_val, y_pred_val)
r2_val = r2_score(y_val, y_pred_val)

print("Validation Results:")
print("Mean Squared Error (MSE):", mse_val)
print("Mean Absolute Error (MAE):", mae_val)
print("R-squared (R2):", r2_val)

Validation Results:
Mean Squared Error (MSE): 1532468379.1302736
Mean Absolute Error (MAE): 1881.513166568991
R-squared (R2): 0.9996998488214139


In [77]:
# Combine true and predicted values into a DataFrame
validation_results = pd.DataFrame({
    "Year": val_data["Year"],
    "Area": val_data["Area"],
    "Products": val_data["Products"],
    "True Export Value": y_val,
    "Predicted Export Value": y_pred_val
})

In [78]:
# Print the validation results
print("Validation Results:")
print(validation_results)

Validation Results:
          Year  Area  Products  True Export Value  Predicted Export Value
2628      2021     0         2       1.038183e+06            1.025115e+06
2629      2021     0         2       1.038183e+06            1.034587e+06
2630      2021     0         2       1.038183e+06            1.035501e+06
2631      2021     0         2       1.038183e+06            1.036068e+06
2632      2021     0         2       1.038183e+06            1.036077e+06
...        ...   ...       ...                ...                     ...
14823120  2022   211        38       7.075696e+04            7.105150e+04
14823121  2022   211        38       7.075696e+04            7.104333e+04
14823122  2022   211        38       7.075696e+04            7.105018e+04
14823123  2022   211        38       7.075696e+04            7.104836e+04
14823124  2022   211        38       7.075696e+04            7.105322e+04

[1236800 rows x 5 columns]


In [79]:
validation_results.to_csv("predictions_results.csv", index=False)
print("Predictionresults saved to validation_results.csv")

Predictionresults saved to validation_results.csv
